# Extract a Shapefile Subset from Excel

## Overview

This tutorial shows you how to use extract a subset from a shapefile using data contained in an Excel spreadsheet.

We will be working with a parcels data layer for the city of San Francisco, California. Given a list of parcel ids in a spreadsheet, we will extract those parcels and save it to another data layer.

**Input Layers**:
* `sf_parcels.zip`: A shapefile of parcels San Francisco
* `parcels_to_export.xlsx`: A spreadsheet containing list of parcels to export.

**Output**:
* `subset.zip`: A zipped shapefile containing a subset of parcels based on the spreadsheet.

**Data Credit**:
* Parcels downloaded from [DataSF Open Data Portal](https://datasf.org/opendata/)

**Video Walkthrough**: [![Watch on YouTube](https://img.shields.io/badge/YouTube-%23FF0000.svg)](https://www.youtube.com/watch?v=p7QzF8kQogo)

## Setup and Data Download

The following blocks of code will install the required packages and download the datasets to your Colab environment.

In [2]:
import os
import pandas as pd
import geopandas as gpd
import zipfile

In [ ]:
data_folder = 'data'
output_folder = 'output'

if not os.path.exists(data_folder):
    os.mkdir(data_folder)
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [ ]:
def download(url):
    filename = os.path.join(data_folder, os.path.basename(url))
    if not os.path.exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

data_url = 'https://github.com/spatialthoughts/geopython-tutorials/releases/download/data/'

download(data_url + 'sf_parcels.zip')
download(data_url + 'parcels_to_export.xlsx')

## Procedure

We first unzip the `sf_parcels.zip` archive and extract the shapefile contained inside. Then we can read it using GeoPandas.

> GeoPandas can read zipped files directly using the `zip://` prefix as described in [Reading and Writing Files](https://geopandas.org/en/stable/docs/user_guide/io.html) section of the documentation. `gpd.read_file('zip:///data/sf_parcels.zip')`. But it was much slower than unzipping and reading the shapefile.

In [ ]:
parcels_filepath = os.path.join(data_folder, 'sf_parcels.zip')

We use Python's built-in `zipfile` module to extract the files in the data directory.

In [ ]:
with zipfile.ZipFile(parcels_filepath) as zf:
  zf.extractall(data_folder)

Once unzipped, we can read the parcels shapefile using GeoPandas.

In [ ]:
parcels_shp = os.path.join(data_folder, 'sf_parcels.shp')
parcels_gdf = gpd.read_file(parcels_shp)

Preview the resulting GeoDataFrame. The parcel ids are contained in the `mapblklot` column.

In [ ]:
parcels_gdf

Next, we read the Excel file containing the parcel ids that we need to export.

In [ ]:
export_file_path = os.path.join(data_folder, 'parcels_to_export.xlsx')

Pandas can read Excel files directly using `read_excel()` function. If you get an error, make sure to install the package `openpyxl` which is used to read excel files.

In [ ]:
export_df = pd.read_excel(export_file_path)
export_df

We need to export all parcels whose ids are given in the `mapblklot` column. We extract that column and create a list.

In [ ]:
id_list = export_df['blklot'].values
id_list

Now we can use Pandas `isin()` method to filter the GeoDataFrame where the `
blklot` column matches any ids from the `id_list`.

In [ ]:
subset_gdf = parcels_gdf[parcels_gdf['blklot'].isin(id_list)]
subset_gdf

We have successfully selected the subset of parcels. We are ready to save the resulting GeoDataFrame as a shapefile. We define the output file path and save the `subset_gdf`.

In [ ]:
output_file = 'subset.shp'
output_path = os.path.join(output_folder, output_file)

In [ ]:
subset_gdf.to_file(output_path)

For ease of data sharing, let's zip all the shapefile parts into a single archive. We again use the `zipfile` module and use the `write()` method to add each sidecar file for the shapefile. The `arcname` parameter is used to avoid creating a sub-folder inside the archive.

In [ ]:
output_zip = 'subset.zip'
output_zip_path = os.path.join(output_folder, output_zip)

with zipfile.ZipFile(output_zip_path, 'w') as output_zf:
  for ext in ['.shp', '.shx', '.prj', '.dbf']:
    filename = 'subset' + ext
    filepath = os.path.join(output_folder, filename)
    output_zf.write(filepath, arcname=filename)